# Kaggle에서 실습할 경우
① Add data 클릭 → ② "metal surface defects" 검색 → ③ Add 클릭

이후 우측 상단 input 폴더를 클릭하여  "neu-metal-surface-defects-data" 가 추가된 것을 확인

# PC에서 실습할 경우
아래 링크로 데이터를 다운로드 받은 후, 주피터 노트북이 저장된 폴더 하위에 'dataset_surface' 폴더를 만들고 압축을 해제해 놓으세요.

데이터셋 : https://www.kaggle.com/datasets/fantacher/neu-metal-surface-defects-data/download

In [ ]:
# 모듈 불러오기
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from glob import glob
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# root dir 설정
root_dir = "../input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data" # 캐글에서 실습할 경우
# root_dir = "dataset_surface" # PC에서 실습할 경우

# os.listdir() 코드로 폴더 탐색
os.listdir(root_dir + "/train")
os.listdir(root_dir + "/train/Inclusion")

In [ ]:
# glob으로 경로명과 함께 출력
glob(root_dir + "/train/Inclusion/*")

In [ ]:
# 폴더명 저장하기 
train_dir = root_dir + "/train/"
val_dir = root_dir + "/valid/"

# 라벨 만들기
labels = os.listdir(train_dir)
labels

# train 하위 폴더별 이미지 개수 파악
for lb in labels:
    print(lb, len(os.listdir(train_dir+lb)))

In [ ]:
# glob으로 이미지데이터 경로 저장하기
paths = glob(train_dir+"*/*")   # train 하위 폴더에 있는 모든 파일의 경로를 paths에 저장
len(paths) # 출력결과 : 1656

In [ ]:
# paths에 저장된 이미지 및 경로 5개 출력
paths[0:5]

In [ ]:
# 첫번째 이미지 불러와서 출력하기
image_pil = Image.open(paths[0])
image_pil

In [ ]:
# array로 변환
image = np.array(image_pil)
print(image.shape)  # 이미지 모양 200 X 200
image

In [ ]:
# array를 이미지로 출력
plt.imshow(image,"gray")
plt.show()

In [ ]:
# 전체 이미지 사이즈 저장
heights = [] # 리스트 생성
widths = []
for path in paths: # 이미지 경로를 하나씩 불러옴
    image_pil = Image.open(path) # 이미지를 하나씩 열기
    image = np.array(image_pil) # array로 변환
    h, w = image.shape # 이미지의 height, width를 h, w에 저장
    heights.append(h) # heights 리스트 변수에 h를 추가
    widths.append(w) # widths 리스트 변수에 w를 추가

# 전체 이미지 사이즈 저장
plt.figure(figsize=(20, 10))
plt.subplot(121)
plt.hist(heights)
plt.title("Heights")
plt.subplot(122)
plt.hist(widths)
plt.title("Widths")
plt.show()

In [ ]:
# 텐서플로우로 이미지 불러오기
gfile = tf.io.read_file(paths[0])
image = tf.io.decode_image(gfile, dtype=tf.float32)
print(image.shape) # 이미지 사이즈  [200,200,1]

# 이미지 출력
plt.imshow(image,"gray")
plt.show()

In [ ]:
# 텐서플로우 데이터제너레이션 (Augmentation)
image.shape # 이미지 사이즈 200,200,1
inputs = image[tf.newaxis, ...] # 앞쪽에 배열 추가 
inputs.shape # 1,200,200,1

# 데이터 제너레이터 객체 생성
datagen = ImageDataGenerator(
    rescale=1./255., # 리스케일링
    zoom_range=0.2, # zoom을 0 ~ 0.2 사이로 랜덤하게 변경
    rotation_range=1, # rotation_range 를 0 ~ 1 사이 랜덤하게 변경
    shear_range=0.5, # shear_range 0 ~ 0.5
    brightness_range=(0,2) # brightness_range 0 ~ 2
)
result = next(iter(datagen.flow(inputs)))  	#실행
plt.subplot(121)
plt.title("Original Image")
plt.imshow(inputs[0, :, :, 0], "gray") # 200x200 출력
plt.subplot(122)
plt.title("Transformed Image")
plt.imshow(result[0, :, :, 0], "gray")
plt.show()